In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malyalam-news-dataset/valid.csv
/kaggle/input/malyalam-news-dataset/train.csv


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

In [3]:
#Read dataset
FILEPATH='/kaggle/input/malyalam-news-dataset'
df_train=pd.read_csv(os.path.join(FILEPATH,'train.csv'))
df_valid=pd.read_csv(os.path.join(FILEPATH,'valid.csv'))

In [4]:
#PREPROCESSING STEPS
from sklearn import preprocessing 
COLUMNS = df_train.columns.str.upper().to_list()
df_train.columns=COLUMNS
df_valid.columns=COLUMNS
label_encoder = preprocessing.LabelEncoder()  
# Encode labels in column 'species'. 
df_train['LABEL']= label_encoder.fit_transform(df_train['LABEL']) 
df_valid['LABEL']= label_encoder.transform(df_valid['LABEL']) 

In [5]:
#Defining the train and validation dataset.
X_train, X_test, y_train, y_test = df_train[['HEADINGS']],df_valid[['HEADINGS']],df_train[['LABEL']],df_valid[['LABEL']]
y_train=np.array(y_train).astype('int').reshape((-1,1))
y_test=np.array(y_test).astype('int').reshape((-1,1))

y_train = tf.keras.utils.to_categorical(y_train)
y_test= tf.keras.utils.to_categorical(y_test)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout

#Custom CallBack and Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=X_train.shape[0]*20,
  decay_rate=1,
  staircase=False)


def tf_model():
    # Input layer
    input_layer = Input(shape=(), dtype=tf.string, name='text_input')

    # Preprocessor layer
    preprocessor_layer = hub.KerasLayer(
        "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
    )
    preprocessed_text = preprocessor_layer(input_layer)

    # Encoder layer
    encoder_layer = hub.KerasLayer(
        "https://tfhub.dev/google/LaBSE/2",
        trainable=False
    )

    embedded_text = encoder_layer(preprocessed_text)["sequence_output"]
    # Pass the embeddings through LSTM layer
    lstm_layer = tf.keras.layers.LSTM(units=64,return_sequences=False)
    lstm_output = lstm_layer(embedded_text)
    # For example, add a Dense layer for classification
    output_layer = layers.Dense(units=3, activation='softmax', name='output')(lstm_output)

    
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Create the model
model = tf_model()

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_word_ids': (None,    0         ['text_input[0][0]']          
                             128),                                                                
                              'input_type_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [7]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['acc']
)

In [8]:
model.fit(X_train,y_train,epochs=15,callbacks=[early_stopping],validation_data=[X_test,y_test],batch_size=1500)

Epoch 1/15
4/4 [==============================] - 1632s 406s/step - loss: 0.6156 - acc: 0.5220 - val_loss: 0.4992 - val_acc: 0.7151
Epoch 2/15
4/4 [==============================] - 1662s 421s/step - loss: 0.4431 - acc: 0.7929 - val_loss: 0.3654 - val_acc: 0.8341
Epoch 3/15
4/4 [==============================] - 1556s 396s/step - loss: 0.3189 - acc: 0.8771 - val_loss: 0.2779 - val_acc: 0.8603
Epoch 4/15
4/4 [==============================] - 1624s 416s/step - loss: 0.2357 - acc: 0.8963 - val_loss: 0.2261 - val_acc: 0.8770
Epoch 5/15
4/4 [==============================] - 1552s 397s/step - loss: 0.1863 - acc: 0.9106 - val_loss: 0.2054 - val_acc: 0.8849
Epoch 6/15
4/4 [==============================] - 1648s 420s/step - loss: 0.1594 - acc: 0.9196 - val_loss: 0.1931 - val_acc: 0.8897
Epoch 7/15
4/4 [==============================] - 1595s 402s/step - loss: 0.1444 - acc: 0.9279 - val_loss: 0.1872 - val_acc: 0.8944
Epoch 8/15
4/4 [==============================] - 1604s 415s/step - loss: 0.

In [9]:
model
